# Analyse des causes d’attrition chez TechNova Partners  
## 1. Ingestion et contrôle qualité des données

**Projet :** HR Analytics – Identification des causes de démission  
**Rôle :** Consultant Data Scientist  
**Objectif du notebook :**  
Construire un DataFrame central propre et cohérent à partir de trois sources RH hétérogènes, en réalisant les premiers contrôles qualité indispensables avant toute analyse exploratoire ou modélisation.


## Contexte et problématique

TechNova Partners fait face à un taux de turnover plus élevé qu’habituellement.  
Le département RH souhaite identifier les causes potentielles de ces départs afin de mettre en place des leviers d’action ciblés.

Trois sources de données internes sont mises à disposition :

1. **SIRH** : informations contractuelles, sociodémographiques et professionnelles
2. **Évaluations annuelles** : performance et satisfaction
3. **Sondage interne** : bien-être, perception, et variable cible de démission

Avant toute analyse ou modélisation, il est indispensable de :
- comprendre la structure de chaque source,
- vérifier la qualité des données,
- identifier les clés de jointure,
- construire un jeu de données central cohérent et anonymisé.


## Démarche méthodologique

La démarche suivie dans ce notebook est la suivante :

1. Chargement des trois fichiers CSV bruts
2. Contrôle qualité initial (dimensions, types, valeurs manquantes, doublons)
3. Harmonisation des clés de jointure
4. Jointure des trois sources
5. Contrôles post-jointure
6. Anonymisation des identifiants employés
7. Sauvegarde d’un dataset central prêt pour l’analyse exploratoire

Les transformations récurrentes sont implémentées dans des fonctions Python réutilisables (dossier `src/`) afin de garantir :
- la reproductibilité,
- la lisibilité,
- et la maintenabilité du projet.


In [1]:
from technova_attrition.config import PATHS
from technova_attrition.data_io import (
    anonymize_employee_id,
    check_duplicates,
    join_sources,
    load_eval,
    load_sirh,
    load_sondage,
)

## Chargement des données brutes

Les fichiers CSV sont stockés dans le dossier `data/raw`.  
Ils correspondent aux extractions brutes fournies par le responsable SIRH.

À ce stade, aucune transformation métier n’est appliquée.


In [2]:
sirh_path = PATHS.data_raw / "extrait_sirh.csv"
eval_path = PATHS.data_raw / "extrait_eval.csv"
sondage_path = PATHS.data_raw / "extrait_sondage.csv"

sirh_df = load_sirh(sirh_path)
eval_df = load_eval(eval_path)
sondage_df = load_sondage(sondage_path)

sirh_df.shape, eval_df.shape, sondage_df.shape

((1470, 12), (1470, 11), (1470, 12))

## Inspection initiale des jeux de données

Nous examinons pour chaque source :
- les dimensions (nombre de lignes et colonnes),
- les types de variables,
- un aperçu des premières lignes.

Cette étape permet de détecter rapidement :
- des incohérences de typage,
- des colonnes inutiles,
- des valeurs manifestement aberrantes.


In [3]:
sirh_df.head()

,id_employee,age,genre,revenu_mensuel,statut_marital,departement,poste,nombre_experiences_precedentes,nombre_heures_travailless,annee_experience_totale,annees_dans_l_entreprise,annees_dans_le_poste_actuel
0,1,41,F,5993,Célibataire,Commercial,Cadre Commercial,8,80,8,6,4
1,2,49,M,5130,Marié(e),Consulting,Assistant de Direction,1,80,10,10,7
2,4,37,M,2090,Célibataire,Consulting,Consultant,6,80,7,0,0
3,5,33,F,2909,Marié(e),Consulting,Assistant de Direction,1,80,8,8,7
4,7,27,M,3468,Marié(e),Consulting,Consultant,9,80,6,2,2


In [4]:
sirh_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   id_employee                     1470 non-null   int64 
 1   age                             1470 non-null   int64 
 2   genre                           1470 non-null   object
 3   revenu_mensuel                  1470 non-null   int64 
 4   statut_marital                  1470 non-null   object
 5   departement                     1470 non-null   object
 6   poste                           1470 non-null   object
 7   nombre_experiences_precedentes  1470 non-null   int64 
 8   nombre_heures_travailless       1470 non-null   int64 
 9   annee_experience_totale         1470 non-null   int64 
 10  annees_dans_l_entreprise        1470 non-null   int64 
 11  annees_dans_le_poste_actuel     1470 non-null   int64 
dtypes: int64(8), object(4)
memory usage: 137.9+ KB


## Recherche de doublons

Avant toute jointure, il est essentiel de vérifier l’unicité des identifiants métiers.

Nous recherchons ici :
- des doublons stricts (lignes identiques),
- des doublons sur les clés de jointure potentielles.

Ces vérifications permettent d’éviter des duplications artificielles lors des jointures.


In [5]:
check_duplicates(sirh_df, subset=["id_employee"]).shape

(0, 12)

In [6]:
check_duplicates(eval_df, subset=["eval_number"]).shape

(0, 11)

In [7]:
check_duplicates(sondage_df, subset=["code_sondage"]).shape

(0, 12)

## Harmonisation des clés de jointure

Les trois sources ne partagent pas une clé strictement identique :

- **SIRH** : `id_employee` (entier)
- **Évaluations** : `eval_number` (ex: "E_1", "E_2", ...)
- **Sondage** : `code_sondage` (entier)

Une transformation est nécessaire pour rendre ces clés compatibles :
- extraction de la partie numérique de `eval_number`
- création d’une clé numérique `eval_number_int`

Cette étape est indispensable pour garantir des jointures fiables.


In [8]:
eval_df[["eval_number", "eval_number_int"]].head()

,eval_number,eval_number_int
0,E_1,1
1,E_2,2
2,E_4,4
3,E_5,5
4,E_7,7


## Jointure des trois sources

Une jointure **inner** est utilisée afin de :
- conserver uniquement les employés présents dans les trois systèmes,
- garantir la cohérence du dataset final.

Cette décision est justifiée dans un contexte d’analyse explicative, où la qualité prime sur l’exhaustivité.


In [9]:
df = join_sources(sirh_df, eval_df, sondage_df, how="inner")
df.shape

(1470, 35)

## Contrôles qualité post-jointure

Après la jointure, nous vérifions :
- l’absence de duplication inattendue,
- la cohérence des dimensions,
- la présence effective de la variable cible.

Ces contrôles permettent de s’assurer que la jointure n’a pas introduit d’erreurs structurelles.


In [10]:
df.duplicated().sum()

np.int64(0)

In [11]:
df.columns

Index(['id_employee', 'age', 'genre', 'revenu_mensuel', 'statut_marital',
       'departement', 'poste', 'nombre_experiences_precedentes',
       'nombre_heures_travailless', 'annee_experience_totale',
       'annees_dans_l_entreprise', 'annees_dans_le_poste_actuel',
       'satisfaction_employee_environnement', 'note_evaluation_precedente',
       'niveau_hierarchique_poste', 'satisfaction_employee_nature_travail',
       'satisfaction_employee_equipe',
       'satisfaction_employee_equilibre_pro_perso', 'eval_number',
       'note_evaluation_actuelle', 'heure_supplementaires',
       'augementation_salaire_precedente', 'eval_number_int',
       'a_quitte_l_entreprise', 'nombre_participation_pee',
       'nb_formations_suivies', 'nombre_employee_sous_responsabilite',
       'code_sondage', 'distance_domicile_travail', 'niveau_education',
       'domaine_etude', 'ayant_enfants', 'frequence_deplacement',
       'annees_depuis_la_derniere_promotion', 'annes_sous_responsable_actuel'],
   

## Anonymisation des identifiants employés

Les données RH sont **hautement sensibles**.  
Avant toute analyse ou modélisation, les identifiants employés doivent être anonymisés.

Méthode utilisée :
- **HMAC-SHA256** avec clé secrète
- anonymisation stable (un même employé garde le même identifiant anonymisé)
- non réversible sans la clé

Cette approche est conforme aux bonnes pratiques RGPD en contexte analytique.


In [12]:
df["employee_id_anon"] = anonymize_employee_id(df["id_employee"])
df = df.drop(columns=["id_employee"])

df[["employee_id_anon"]].head()

,employee_id_anon
0,emp_02593e2d3346ef8d
1,emp_6e1ebb9e806c698d
2,emp_db8fbd41f5eb0dcb
3,emp_dc65e725b0822162
4,emp_6525529cce69bce2


## Sauvegarde du dataset central

Le DataFrame final est sauvegardé dans le dossier `data/processed`.

Ce fichier constitue :
- l’entrée unique pour l’analyse exploratoire,
- la base de travail pour le feature engineering,
- la référence utilisée pour la partie SQL et la modélisation.


In [13]:
output_path = PATHS.data_processed / "employees_joined.parquet"
df.to_parquet(output_path, index=False)

output_path

WindowsPath('G:/Mon Drive/OC/Projet_4/technova_attrition/data/processed/employees_joined.parquet')

## Conclusion de l’étape d’ingestion

À l’issue de ce notebook, nous disposons :
- d’un dataset central issu de trois sources RH,
- de données contrôlées et cohérentes,
- d’identifiants employés anonymisés,
- d’un fichier prêt pour l’analyse exploratoire et statistique.

La prochaine étape consistera à :
- analyser les distributions des variables,
- comparer les employés ayant quitté l’entreprise à ceux encore présents,
- identifier les premiers signaux explicatifs de l’attrition.
